## setup libraries

In [1]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt data for a given date range

Note: this example only fetches the events table. The gdelt python API also can pull from the mentions and gkg tables. See https://linwoodc3.github.io/gdeltPyR/ for full API details



In [12]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

In [13]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()
lst_data = []

# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)


def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='gkg',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
          return filename
    except:
        print("Error occurred")

# test gkg table
#疫情开始前
test_data1 = [get_filename(x) for x in pd.date_range('2019 Feb 1','2019 May 1')]
#2020年2月11日，世界卫生组织正式宣布将此疾病定名为“2019冠状病毒病”
test_data2 = [get_filename(x) for x in pd.date_range('2020 Feb 1','2020 May 1')]
#疫情仍在继续
test_data3 = [get_filename(x) for x in pd.date_range('2021 Feb 1','2021 May 1')]

dbg(test_data3)
#downloading
Before_Epidemics = list(e.map(intofile,test_data1))
Start_Epidemics = list(e.map(intofile,test_data2))
Still_Epidemics = list(e.map(intofile,test_data3))

dbg(Still_Epidemics)

['20210201_gdeltdata.csv', '20210202_gdeltdata.csv', '20210203_gdeltdata.csv', '20210204_gdeltdata.csv', '20210205_gdeltdata.csv', '20210206_gdeltdata.csv', '20210207_gdeltdata.csv', '20210208_gdeltdata.csv', '20210209_gdeltdata.csv', '20210210_gdeltdata.csv', '20210211_gdeltdata.csv', '20210212_gdeltdata.csv', '20210213_gdeltdata.csv', '20210214_gdeltdata.csv', '20210215_gdeltdata.csv', '20210216_gdeltdata.csv', '20210217_gdeltdata.csv', '20210218_gdeltdata.csv', '20210219_gdeltdata.csv', '20210220_gdeltdata.csv', '20210221_gdeltdata.csv', '20210222_gdeltdata.csv', '20210223_gdeltdata.csv', '20210224_gdeltdata.csv', '20210225_gdeltdata.csv', '20210226_gdeltdata.csv', '20210227_gdeltdata.csv', '20210228_gdeltdata.csv', '20210301_gdeltdata.csv', '20210302_gdeltdata.csv', '20210303_gdeltdata.csv', '20210304_gdeltdata.csv', '20210305_gdeltdata.csv', '20210306_gdeltdata.csv', '20210307_gdeltdata.csv', '20210308_gdeltdata.csv', '20210309_gdeltdata.csv', '20210310_gdeltdata.csv', '20210311_g

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


...

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

Error occurred


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_ba

['20210201_gdeltdata.csv', '20210202_gdeltdata.csv', '20210203_gdeltdata.csv', '20210204_gdeltdata.csv', '20210205_gdeltdata.csv', '20210206_gdeltdata.csv', '20210207_gdeltdata.csv', '20210208_gdeltdata.csv', '20210209_gdeltdata.csv', '20210210_gdeltdata.csv', '20210211_gdeltdata.csv', '20210212_gdeltdata.csv', '20210213_gdeltdata.csv', '20210214_gdeltdata.csv', '20210215_gdeltdata.csv', '20210216_gdeltdata.csv', '20210217_gdeltdata.csv', '20210218_gdeltdata.csv', '20210219_gdeltdata.csv', '20210220_gdeltdata.csv', '20210221_gdeltdata.csv', None, '20210223_gdeltdata.csv', '20210224_gdeltdata.csv', '20210225_gdeltdata.csv', '20210226_gdeltdata.csv', '20210227_gdeltdata.csv', '20210228_gdeltdata.csv', '20210301_gdeltdata.csv', '20210302_gdeltdata.csv', '20210303_gdeltdata.csv', '20210304_gdeltdata.csv', '20210305_gdeltdata.csv', '20210306_gdeltdata.csv', '20210307_gdeltdata.csv', '20210308_gdeltdata.csv', None, '20210310_gdeltdata.csv', '20210311_gdeltdata.csv', '20210312_gdeltdata.csv',

In [14]:
#!zip -o articles.zip *.csv

In [15]:
def find_date(results):
#寻找有用的日期
  result = []
  for i in results:
    if i != None:
      result.append(i)
  return result

result_Before = find_date(Before_Epidemics)
result_Start = find_date(Start_Epidemics)
result_Still = find_date(Still_Epidemics)
print("Before >>>>>>>>>>    ", result_Before)
print("Start >>>>>>>>>>>    ", result_Start)
print("Still >>>>>>>>>>>    ", result_Still)

Before >>>>>>>>>>     ['20190201_gdeltdata.csv', '20190202_gdeltdata.csv', '20190203_gdeltdata.csv', '20190204_gdeltdata.csv', '20190205_gdeltdata.csv', '20190206_gdeltdata.csv', '20190207_gdeltdata.csv', '20190208_gdeltdata.csv', '20190209_gdeltdata.csv', '20190210_gdeltdata.csv', '20190211_gdeltdata.csv', '20190212_gdeltdata.csv', '20190213_gdeltdata.csv', '20190215_gdeltdata.csv', '20190216_gdeltdata.csv', '20190217_gdeltdata.csv', '20190218_gdeltdata.csv', '20190219_gdeltdata.csv', '20190220_gdeltdata.csv', '20190221_gdeltdata.csv', '20190222_gdeltdata.csv', '20190223_gdeltdata.csv', '20190224_gdeltdata.csv', '20190225_gdeltdata.csv', '20190226_gdeltdata.csv', '20190227_gdeltdata.csv', '20190228_gdeltdata.csv', '20190301_gdeltdata.csv', '20190302_gdeltdata.csv', '20190303_gdeltdata.csv', '20190304_gdeltdata.csv', '20190305_gdeltdata.csv', '20190306_gdeltdata.csv', '20190307_gdeltdata.csv', '20190308_gdeltdata.csv', '20190309_gdeltdata.csv', '20190310_gdeltdata.csv', '20190311_gdelt

## read downloaded files into RDDs

In [16]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

test_data1 = sqlContext.read.option("header", "true").csv(result_Before)
test_data2 = sqlContext.read.option("header", "true").csv(result_Start)
test_data3 = sqlContext.read.option("header", "true").csv(result_Still)

#dbg(test_data)
test_data_rdd1 = test_data1.rdd
test_data_rdd2 = test_data2.rdd
test_data_rdd3 = test_data3.rdd


dbg(test_data_rdd1)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[('20190225234500-0', '20190225234500'), ('20190225234500-1', '20190225234500'), ('20190225234500-2', '20190225234500'), ('20190225234500-3', '20190225234500'), ('20190225234500-4', '20190225234500'), ('20190225234500-5', '20190225234500'), ('20190225234500-6', '20190225234500'), ('20190225234500-7', '20190225234500'), ('20190225234500-8', '20190225234500'), ('20190225234500-9', '20190225234500'), ('20190225234500-10', '20190225234500'), ('20190225234500-11', '20190225234500'), ('20190225234500-12', '20190225234500'), ('20190225234500-13', '20190225234500'), ('20190225234500-14', '20190225234500'), ('20190225234500-15', '20190225234500'), ('20190225234500-16', '20190225234500'), ('20190225234500-17', '20190225234500'), ('20190225234500-18', '20190225234500'), ('20190225234500-19', '20190225234500'), ('20190225234500-20', '20190225234500'), ('20190225234500-21', '20190225234500'), ('20190225234500-22', '20190225234500'), ('20190225234500-23', '20190225234500'), ('20190225234500-24', '20

In [17]:
test_data1.show()

+-----------------+--------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+
|      GKGRECORDID|          DATE|SourceCollectionIdentifier|    SourceCommonName|  DocumentIdentifier|              Counts|            V2Counts|              Themes|            V2Themes|           Locations|         V2Locations|             Persons|           V2Persons|       Organizations|     V2Organizations|              V2Tone|               Dates|                GCAM|        SharingImage|       RelatedImages|   SocialImageEmbeds|

## countryName_And_Theme

> RDD that countain all the country fields and themes fields without split

# Country_And_Theme

> this RDD contain countryCode with a list Theme with each artical




In [18]:
#take country and theme as rdd 
countryName_And_Theme1 = test_data_rdd1.map(lambda a: (a.Locations, a.Themes))
countryName_And_Theme2 = test_data_rdd2.map(lambda a: (a.Locations, a.Themes))
countryName_And_Theme3 = test_data_rdd3.map(lambda a: (a.Locations, a.Themes))

dbg(countryName_And_Theme1)

[('2#New York, United States#US#USNY#42.1497#-74.9384#NY;1#Canada#CA#CA#60#-96#CA;2#New Jersey, United States#US#USNJ#40.314#-74.5089#NJ;2#Massachusetts, United States#US#USMA#42.2373#-71.5314#MA', 'ARMEDCONFLICT;LEGALIZE;USPEC_POLICY1;EPU_POLICY;EPU_POLICY_BUDGET;ALLIANCE;GENERAL_HEALTH;MEDICAL;CRISISLEX_C03_WELLBEING_HEALTH;CRIME_ILLEGAL_DRUGS;LEADER;TAX_FNCACT;TAX_FNCACT_LAWMAKERS;USPEC_POLITICS_GENERAL1;EPU_POLICY_LAWMAKERS;MEDIA_MSM;EPU_ECONOMY_HISTORIC;ECON_TAXATION;EPU_POLICY_TAX;EPU_CATS_TAXES;TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_MASSACHUSETTS;LEGISLATION;EPU_POLICY_LEGISLATION;TAX_FNCACT_STRINGER;TAX_FNCACT_GOVERNOR;TAX_FNCACT_LEADER;TAX_FNCACT_PRESIDENT;TAX_FNCACT_LEADERS;WB_1921_PRIVATE_SECTOR_DEVELOPMENT;WB_405_BUSINESS_CLIMATE;WB_2531_INSPECTIONS_LICENSING_AND_PERMITS;WB_2530_BUSINESS_ENVIRONMENT;BAN;WB_1331_HEALTH_TECHNOLOGIES;WB_2453_ORGANIZED_CRIME;WB_1350_PHARMACEUTICALS;WB_2433_CONFLICT_AND_VIOLENCE;WB_621_HEALTH_NUTRITION_AND_POPULATION;WB_2432_FRAGILITY_CONFLICT_AN

In [19]:
#make country as a key
def find_country(string):
  """spilt the str and find the country"""
  if string == None:
    return str("Not found")
  lst = string.split('#')
  if len(lst) >= 3:
    return lst[2]
  else:
    return str("Not found")
take_country1 = countryName_And_Theme1.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")
take_country2 = countryName_And_Theme2.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")
take_country3 = countryName_And_Theme3.map(lambda a: (find_country(a[0]), a[1])).filter(lambda a: a[0] != "Not found")

dbg(take_country1)

[('US', 'ARMEDCONFLICT;LEGALIZE;USPEC_POLICY1;EPU_POLICY;EPU_POLICY_BUDGET;ALLIANCE;GENERAL_HEALTH;MEDICAL;CRISISLEX_C03_WELLBEING_HEALTH;CRIME_ILLEGAL_DRUGS;LEADER;TAX_FNCACT;TAX_FNCACT_LAWMAKERS;USPEC_POLITICS_GENERAL1;EPU_POLICY_LAWMAKERS;MEDIA_MSM;EPU_ECONOMY_HISTORIC;ECON_TAXATION;EPU_POLICY_TAX;EPU_CATS_TAXES;TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_MASSACHUSETTS;LEGISLATION;EPU_POLICY_LEGISLATION;TAX_FNCACT_STRINGER;TAX_FNCACT_GOVERNOR;TAX_FNCACT_LEADER;TAX_FNCACT_PRESIDENT;TAX_FNCACT_LEADERS;WB_1921_PRIVATE_SECTOR_DEVELOPMENT;WB_405_BUSINESS_CLIMATE;WB_2531_INSPECTIONS_LICENSING_AND_PERMITS;WB_2530_BUSINESS_ENVIRONMENT;BAN;WB_1331_HEALTH_TECHNOLOGIES;WB_2453_ORGANIZED_CRIME;WB_1350_PHARMACEUTICALS;WB_2433_CONFLICT_AND_VIOLENCE;WB_621_HEALTH_NUTRITION_AND_POPULATION;WB_2432_FRAGILITY_CONFLICT_AND_VIOLENCE;WB_2456_DRUGS_AND_NARCOTICS;TAX_FNCACT_SPEAKER;TAX_FNCACT_MAJORITY_LEADER;WB_696_PUBLIC_SECTOR_MANAGEMENT;WB_840_JUSTICE;JUSTICE;CRISISLEX_CRISISLEXREC;EPU_POLICY_LAW;SECURITY_SER

In [20]:
#make theme as a list
def to_list(string):
  if string == None:
    return str("No Theme")
  else:
    res = string.split(";")
    return res[:-1]
Country_And_Theme1 = take_country1.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")
Country_And_Theme2 = take_country2.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")
Country_And_Theme3 = take_country3.map(lambda a: (a[0], to_list(a[1]))).filter(lambda a: a[1] != "No Theme")

dbg(Country_And_Theme1)

[('US', ['ARMEDCONFLICT', 'LEGALIZE', 'USPEC_POLICY1', 'EPU_POLICY', 'EPU_POLICY_BUDGET', 'ALLIANCE', 'GENERAL_HEALTH', 'MEDICAL', 'CRISISLEX_C03_WELLBEING_HEALTH', 'CRIME_ILLEGAL_DRUGS', 'LEADER', 'TAX_FNCACT', 'TAX_FNCACT_LAWMAKERS', 'USPEC_POLITICS_GENERAL1', 'EPU_POLICY_LAWMAKERS', 'MEDIA_MSM', 'EPU_ECONOMY_HISTORIC', 'ECON_TAXATION', 'EPU_POLICY_TAX', 'EPU_CATS_TAXES', 'TAX_WORLDLANGUAGES', 'TAX_WORLDLANGUAGES_MASSACHUSETTS', 'LEGISLATION', 'EPU_POLICY_LEGISLATION', 'TAX_FNCACT_STRINGER', 'TAX_FNCACT_GOVERNOR', 'TAX_FNCACT_LEADER', 'TAX_FNCACT_PRESIDENT', 'TAX_FNCACT_LEADERS', 'WB_1921_PRIVATE_SECTOR_DEVELOPMENT', 'WB_405_BUSINESS_CLIMATE', 'WB_2531_INSPECTIONS_LICENSING_AND_PERMITS', 'WB_2530_BUSINESS_ENVIRONMENT', 'BAN', 'WB_1331_HEALTH_TECHNOLOGIES', 'WB_2453_ORGANIZED_CRIME', 'WB_1350_PHARMACEUTICALS', 'WB_2433_CONFLICT_AND_VIOLENCE', 'WB_621_HEALTH_NUTRITION_AND_POPULATION', 'WB_2432_FRAGILITY_CONFLICT_AND_VIOLENCE', 'WB_2456_DRUGS_AND_NARCOTICS', 'TAX_FNCACT_SPEAKER', 'TAX_F

In [21]:
type(Country_And_Theme1)

pyspark.rdd.PipelinedRDD

# Start to anlysis

#   ** function for finding Frequent_Itemsets**



## def Frequent_Itemsets(countryCode):

> input: countryCode for country to Analysis

> ouptut: individual, apr_stage2, n

> ***Return:***
*   individual : item with Number of occurrences in basket
*   apr_stage2 ： item pairs Number of occurrences in basket
*   n ： The whole size of a basket







In [22]:

#put it into function

def Frequent_Itemsets(Country_And_Theme, countryCode):
  """remember to change the frequent_num"""
  frequent_num = 50 #define at least how many occurrences are considered frequent

  Themes_of_Country = Country_And_Theme.filter(lambda a: a[0] == countryCode).map(lambda a: a[1])
  n = Themes_of_Country.count()
  #find frequence set and broadcast
  individual = Themes_of_Country.flatMap(lambda a:[(i,1) for i in a]).reduceByKey(lambda a,b:a+b)
  individual1 = individual.filter(lambda a: a[1] >= frequent_num).collectAsMap()   #change the frequence here !!!!!!!!
  individual_broad = sc.broadcast(individual1)
  def select(line):
    res = []
    for i,word1 in enumerate(line):
      if word1 in individual_broad.value:
        for j, word2 in enumerate(line):
          if i > j and word2 in individual_broad.value:
            res.append(tuple(sorted([word1, word2])))
    return res
  #find frequence pairs
  apr_stage2 = Themes_of_Country.flatMap(lambda a:select(a)).map(lambda a:(a, 1)).reduceByKey(lambda a,b:a+b)
  return individual, apr_stage2, n, individual_broad

#individual_Pairs_CH, Frequent_Themes_in_CH, count_of_total_pairs, individual_broad= Frequent_Itemsets("CH")
#dbg(individual_Pairs_CH)
#dbg(Frequent_Themes_in_CH)
#dbg(count_of_total_pairs)

In [23]:
individual_Pairs_CH1, Frequent_Themes_in_CH1, count_of_total_pairs_CH1, individual_broad_CH1= Frequent_Itemsets(Country_And_Theme1, "CH")
individual_Pairs_CH2, Frequent_Themes_in_CH2, count_of_total_pairs_CH2, individual_broad_CH2= Frequent_Itemsets(Country_And_Theme2, "CH")
individual_Pairs_CH3, Frequent_Themes_in_CH3, count_of_total_pairs_CH3, individual_broad_CH3= Frequent_Itemsets(Country_And_Theme3, "CH")


# 2. computing the support， confidence and Interest rate


## def support(Frequent_pairs , n):

> Frequent_pairs: item Pairs with Number of occurrences in basket

> total count of Baskets items of the country
  
> ***Return***: support rate 


## def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):

> individual_pairs: item with Number of occurrences in basket

> Frequent_Themes： item pairs Number of occurrences in basket

> total count of Baskets items of the country

> ***Return***: confidence

## def interest_rate(individual, confidence):

> individual : item with Number of occurrences in basket

> confidence : Pairs with confidence rate

> ***Return***: interest_rate






In [24]:
def support(Frequent_pairs , n):
  """/"""
  Support_For_Pairs = Frequent_pairs.map(lambda a:(a[0], a[1]/n))
  return Support_For_Pairs
support_CH1 = support(Frequent_Themes_in_CH1, count_of_total_pairs_CH1)
support_CH2 = support(Frequent_Themes_in_CH2, count_of_total_pairs_CH2)
support_CH3 = support(Frequent_Themes_in_CH3, count_of_total_pairs_CH3)

dbg(support_CH1)
dbg(support_CH2)
dbg(support_CH3)

[(('CRISISLEX_C03_WELLBEING_HEALTH', 'EDUCATION'), 0.04521739130434783), (('CRISISLEX_CRISISLEXREC', 'SCIENCE'), 0.044968944099378884), (('EDUCATION', 'WB_1350_PHARMACEUTICALS'), 0.016149068322981366), (('SOC_POINTSOFINTEREST_UNIVERSITY', 'WB_1350_PHARMACEUTICALS'), 0.011428571428571429), (('SOC_POINTSOFINTEREST_UNIVERSITY', 'TAX_ETHNICITY_CHINESE'), 0.042732919254658386), (('SOC_POINTSOFINTEREST_UNIVERSITY', 'TAX_WORLDLANGUAGES_CHINESE'), 0.042732919254658386), (('CRISISLEX_C03_WELLBEING_HEALTH', 'TAX_FNCACT_AUTHOR'), 0.00422360248447205), (('TAX_FNCACT', 'TAX_WORLDMAMMALS'), 0.17391304347826086), (('SCIENCE', 'TAX_WORLDMAMMALS'), 0.02732919254658385), (('EDUCATION', 'TAX_WORLDMAMMALS_HUMAN'), 0.035527950310559005), (('SOC_POINTSOFINTEREST_UNIVERSITY', 'TAX_WORLDMAMMALS_HUMAN'), 0.020124223602484472), (('TAX_ETHNICITY', 'TAX_WORLDLANGUAGES_ENGLISH'), 0.023354037267080744), (('TAX_WORLDLANGUAGES_ENGLISH', 'WB_1331_HEALTH_TECHNOLOGIES'), 0.0012422360248447205), (('TAX_ETHNICITY', 'TAX_T

In [25]:
def confidence(individual_pairs, Frequent_Themes, count_of_total_pairs):
  """..."""
  sup_individual = individual_pairs.map(lambda a:(a[0], a[1]/count_of_total_pairs))
  sup_ij = Frequent_Themes.map(lambda a:(a[0][0], (a[0][1], a[1]/count_of_total_pairs)))
  sup_ji = Frequent_Themes.map(lambda a:(a[0][1], (a[0][0], a[1]/count_of_total_pairs)))
  sup_all_ij_and_ji = sup_ij.union(sup_ji)
  # dbg(sup_all_ij_and_ji)
  all = sup_all_ij_and_ji.join(sup_individual)
  # dbg(all)
  confidence = all.map(lambda a: ((a[0], a[1][0][0]), a[1][0][1]/a[1][1]))
  return confidence

confidence_CH1 = confidence(individual_Pairs_CH1, Frequent_Themes_in_CH1, count_of_total_pairs_CH1)
confidence_CH2 = confidence(individual_Pairs_CH2, Frequent_Themes_in_CH2, count_of_total_pairs_CH2)
confidence_CH3 = confidence(individual_Pairs_CH3, Frequent_Themes_in_CH3, count_of_total_pairs_CH3)


dbg(confidence_CH1)

[(('NATURAL_DISASTER', 'TAX_FNCACT_PRESIDENT'), 0.2548262548262548), (('NATURAL_DISASTER', 'TAX_ETHNICITY'), 0.7760617760617761), (('NATURAL_DISASTER', 'TAX_ETHNICITY_ASIAN'), 0.0694980694980695), (('NATURAL_DISASTER', 'WB_2453_ORGANIZED_CRIME'), 0.06177606177606178), (('NATURAL_DISASTER', 'WB_167_PORTS'), 0.02702702702702703), (('NATURAL_DISASTER', 'USPEC_UNCERTAINTY1'), 0.12355212355212356), (('NATURAL_DISASTER', 'TAX_FNCACT_FARMERS'), 0.08494208494208494), (('NATURAL_DISASTER', 'WB_1235_CENTRAL_BANKS'), 0.03861003861003861), (('NATURAL_DISASTER', 'TAX_FNCACT_SCIENTIST'), 0.0694980694980695), (('NATURAL_DISASTER', 'SOC_EMERGINGTECH'), 0.03474903474903475), (('NATURAL_DISASTER', 'SANCTIONS'), 0.04633204633204634), (('NATURAL_DISASTER', 'TAX_FNCACT_PRODUCER'), 0.02702702702702703), (('NATURAL_DISASTER', 'WB_697_SOCIAL_PROTECTION_AND_LABOR'), 0.11969111969111969), (('NATURAL_DISASTER', 'TAX_AIDGROUPS'), 0.019305019305019305), (('NATURAL_DISASTER', 'TAX_FNCACT_CHAIRMAN'), 0.0386100386100

In [26]:
def interest_rate(individual, confidence, n):
  individual_sup = individual.map(lambda a:(a[0], a[1]/n))
  interest_join = confidence.map(lambda a:(a[0][1], a)).join(individual_sup)
  interest = interest_join.map(lambda a: (a[1][0][0], a[1][0][1] - a[1][1]))
  return interest

interest_CH1  = interest_rate(individual_Pairs_CH1, confidence_CH1, count_of_total_pairs_CH1)
interest_CH2  = interest_rate(individual_Pairs_CH2, confidence_CH2, count_of_total_pairs_CH2)
interest_CH3  = interest_rate(individual_Pairs_CH3, confidence_CH3, count_of_total_pairs_CH3)


dbg(interest_CH1)
dbg(interest_CH2)
dbg(interest_CH3)

[(('NATURAL_DISASTER', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), 0.01818364948799732), (('TAX_FNCACT_SUPPORTERS', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), -0.022136960452865794), (('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), -0.03206880076445294), (('URBAN', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), 0.015046583850931675), (('EPU_CATS_TAXES', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), -0.01813745523701621), (('WB_525_RENEWABLE_ENERGY', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), -0.02858549162076644), (('TAX_FNCACT_TROOPS', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), -0.011089779785431957), (('RURAL', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), 0.013152644456992282), (('TAX_WORLDLANGUAGES_RUSSIA', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), -0.02867407342606363), (('WB_548_PPP_IN_OIL_AND_GAS', 'WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY'), 0.13436476566911348), (('TAX_ETHNICITY_RUSSIAN', 'WB_1427_NON_COMM

In [32]:
dbg(individual_Pairs_CH1.takeOrdered(10, lambda a: -a[1]))
dbg(individual_Pairs_CH2.takeOrdered(10, lambda a: -a[1]))
dbg(individual_Pairs_CH3.takeOrdered(10, lambda a: -a[1]))

[('TAX_FNCACT', 3648), ('TAX_ETHNICITY', 2713), ('TAX_WORLDLANGUAGES', 2382), ('EPU_POLICY', 2270), ('EPU_ECONOMY_HISTORIC', 1902), ('TAX_ETHNICITY_CHINESE', 1634), ('TAX_WORLDLANGUAGES_CHINESE', 1634), ('USPEC_POLITICS_GENERAL1', 1554), ('LEADER', 1494), ('UNGP_FORESTS_RIVERS_OCEANS', 1451)]
[('TAX_FNCACT', 6292), ('TAX_DISEASE', 5416), ('WB_621_HEALTH_NUTRITION_AND_POPULATION', 5018), ('TAX_DISEASE_CORONAVIRUS', 4831), ('GENERAL_HEALTH', 4789), ('WB_635_PUBLIC_HEALTH', 4383), ('WB_2165_HEALTH_EMERGENCIES', 4100), ('MEDICAL', 4093), ('TAX_ETHNICITY', 4049), ('WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE', 4037)]
[('TAX_FNCACT', 2658), ('TAX_ETHNICITY', 2037), ('TAX_WORLDLANGUAGES', 1749), ('EPU_POLICY', 1672), ('WB_621_HEALTH_NUTRITION_AND_POPULATION', 1394), ('GENERAL_HEALTH', 1382), ('USPEC_POLITICS_GENERAL1', 1260), ('CRISISLEX_CRISISLEXREC', 1243), ('TAX_ETHNICITY_CHINESE', 1224), ('TAX_WORLDLANGUAGES_CHINESE', 1224)]


In [27]:
dbg(support_CH1.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_CH1.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH1.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH1.takeOrdered(5,lambda a:-a[1]))

[(('TAX_ETHNICITY', 'TAX_FNCACT'), 0.6437267080745341), (('TAX_FNCACT', 'TAX_WORLDLANGUAGES'), 0.5639751552795031), (('TAX_ETHNICITY', 'TAX_WORLDLANGUAGES'), 0.5515527950310559), (('EPU_POLICY', 'TAX_FNCACT'), 0.5485714285714286), (('EPU_ECONOMY_HISTORIC', 'TAX_FNCACT'), 0.4477018633540373)]
[(('ECON_FREETRADE', 'WB_698_TRADE'), 1.0), (('ECON_FREETRADE', 'TAX_FNCACT'), 1.0), (('TAX_FNCACT_SUPPORTERS', 'TAX_FNCACT'), 1.0), (('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 'TAX_FNCACT_OFFICER'), 1.0), (('TAX_FNCACT_CHIEF_FINANCIAL_OFFICER', 'TAX_MILITARY_TITLE_OFFICER'), 1.0)]
[(('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_FNCACT'), -0.5165048952521318), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_ETHNICITY'), -0.3859016738604063), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_WORLDLANGUAGES'), -0.35451310664280444), (('TAX_RELIGION_CHURCH', 'EPU_ECONOMY_HISTORIC'), -0.32160318762451656), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'EPU_ECONOMY_HISTORIC'), -0.32000421096957576)]

In [28]:
dbg(support_CH2.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_CH2.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH2.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH2.takeOrdered(5,lambda a:-a[1]))

[(('TAX_DISEASE', 'TAX_FNCACT'), 0.7206050228310502), (('TAX_DISEASE', 'TAX_DISEASE_CORONAVIRUS'), 0.6893550228310502), (('TAX_DISEASE', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.6793664383561644), (('TAX_FNCACT', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.678082191780822), (('GENERAL_HEALTH', 'TAX_FNCACT'), 0.6484018264840182)]
[(('EPU_POLICY_CENTRAL_BANK', 'USPEC_POLICY1'), 1.0), (('EPU_POLICY_CENTRAL_BANK', 'WB_1920_FINANCIAL_SECTOR_DEVELOPMENT'), 1.0), (('EPU_POLICY_CENTRAL_BANK', 'EPU_CATS_MONETARY_POLICY'), 1.0), (('EPU_POLICY_CENTRAL_BANK', 'EPU_POLICY'), 1.0), (('TAX_ETHNICITY_CANADIANS', 'TAX_FNCACT'), 1.0)]
[(('TAX_FNCACT_PROSECUTORS', 'TAX_DISEASE'), -0.49746873138773084), (('TAX_FNCACT_PROSECUTORS', 'TAX_DISEASE_CORONAVIRUS'), -0.4864564721064125), (('WB_1039_PROPERTY_LAWS_AND_REGULATIONS', 'TAX_DISEASE_CORONAVIRUS'), -0.44649787997390733), (('WB_1917_INTELLECTUAL_PROPERTY', 'TAX_DISEASE_CORONAVIRUS'), -0.4434533834867879), (('TAX_FNCACT_OPERATOR', 'TAX_DISEASE_CORONAV

In [29]:
dbg(support_CH3.takeOrdered(5, lambda a:-a[1]))
dbg(confidence_CH3.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH3.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH3.takeOrdered(5,lambda a:-a[1]))

[(('TAX_ETHNICITY', 'TAX_FNCACT'), 0.6434316353887399), (('TAX_FNCACT', 'TAX_WORLDLANGUAGES'), 0.55328418230563), (('TAX_ETHNICITY', 'TAX_WORLDLANGUAGES'), 0.5529490616621984), (('EPU_POLICY', 'TAX_FNCACT'), 0.5465817694369973), (('TAX_FNCACT', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 0.44470509383378015)]
[(('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 'WB_621_HEALTH_NUTRITION_AND_POPULATION'), 1.0), (('WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY', 'WB_1406_DISEASES'), 1.0), (('TAX_FNCACT_VICE_PRESIDENT', 'USPEC_POLITICS_GENERAL1'), 1.0), (('TAX_FNCACT_VICE_PRESIDENT', 'TAX_FNCACT_PRESIDENT'), 1.0), (('TAX_FNCACT_VICE_PRESIDENT', 'TAX_FNCACT'), 1.0)]
[(('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_FNCACT'), -0.35343723740546595), (('HUMAN_RIGHTS_ABUSES', 'UNGP_HEALTHCARE'), -0.3067041314360349), (('HUMAN_RIGHTS_ABUSES', 'WB_635_PUBLIC_HEALTH'), -0.30486526431566646), (('EPU_POLICY_CENTRAL_BANK', 'TAX_ETHNICITY_CHINESE'), -0.30304481041746456), (('EPU_POLICY_CENTRAL_BANK', 'TAX

In [33]:
dbg(interest_CH1.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH2.takeOrdered(5,lambda a:-a[1]))
dbg(interest_CH3.takeOrdered(5,lambda a:-a[1]))

[(('WB_707_POVERTY_MEASUREMENT_AND_ANALYSIS', 'WB_1152_ECONOMIC_AND_SOCIAL_MOBILITY'), 0.9873291925465838), (('WB_1152_ECONOMIC_AND_SOCIAL_MOBILITY', 'WB_707_POVERTY_MEASUREMENT_AND_ANALYSIS'), 0.9873291925465838), (('TAX_FNCACT_CAPTAIN', 'TAX_MILITARY_TITLE_CAPTAIN'), 0.986832298136646), (('TAX_MILITARY_TITLE_CAPTAIN', 'TAX_FNCACT_CAPTAIN'), 0.986832298136646), (('TAX_RELIGION_MUSLIM', 'TAX_ETHNICITY_MUSLIM'), 0.986583850931677)]
[(('TAX_MILITARY_TITLE_SOLDIERS', 'TAX_FNCACT_SOLDIERS'), 0.992579908675799), (('TAX_MILITARY_TITLE_COMMANDER', 'TAX_FNCACT_COMMANDER'), 0.9924372146118722), (('TAX_WORLDLANGUAGES_GERMAN', 'TAX_ETHNICITY_GERMAN'), 0.9921518264840182), (('TAX_ETHNICITY_GERMAN', 'TAX_WORLDLANGUAGES_GERMAN'), 0.9921518264840182), (('WB_2120_SATELLITES', 'WB_2329_ACCESS_AND_CONNECTIVITY'), 0.9920091324200914)]
[(('TAX_WORLDLANGUAGES_ENGLISH', 'TAX_ETHNICITY_ENGLISH'), 0.9829088471849866), (('TAX_ETHNICITY_ENGLISH', 'TAX_WORLDLANGUAGES_ENGLISH'), 0.9829088471849866), (('WB_2601_TR

In [34]:
dbg(interest_CH1.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH2.takeOrdered(5,lambda a:a[1]))
dbg(interest_CH3.takeOrdered(5,lambda a:a[1]))

[(('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_FNCACT'), -0.5165048952521318), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_ETHNICITY'), -0.3859016738604063), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_WORLDLANGUAGES'), -0.35451310664280444), (('TAX_RELIGION_CHURCH', 'EPU_ECONOMY_HISTORIC'), -0.32160318762451656), (('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'EPU_ECONOMY_HISTORIC'), -0.32000421096957576)]
[(('TAX_FNCACT_PROSECUTORS', 'TAX_DISEASE'), -0.49746873138773084), (('TAX_FNCACT_PROSECUTORS', 'TAX_DISEASE_CORONAVIRUS'), -0.4864564721064125), (('WB_1039_PROPERTY_LAWS_AND_REGULATIONS', 'TAX_DISEASE_CORONAVIRUS'), -0.44649787997390733), (('WB_1917_INTELLECTUAL_PROPERTY', 'TAX_DISEASE_CORONAVIRUS'), -0.4434533834867879), (('TAX_FNCACT_OPERATOR', 'TAX_DISEASE_CORONAVIRUS'), -0.4393550228310502)]
[(('WB_2372_AUTHENTICATION_AND_AUTHORIZATION', 'TAX_FNCACT'), -0.35343723740546595), (('HUMAN_RIGHTS_ABUSES', 'UNGP_HEALTHCARE'), -0.3067041314360349), (('HUMAN_RIGHTS_ABU